In [1]:
import keras
from keras.datasets import imdb

Using TensorFlow backend.
/Users/alvinjin/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


## IMDB dataset
50,000 reviews from the Internet Movie Database. 50% positive reviews.
Each review (a sequence of words) have been turned into a sequence of integers, 
which are the indices of words in a dictionary. 
Only keep the top 10,000 most frequently occurring words in the training data with the largest index as 9999.

In [2]:
(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=10000)

In [3]:
train_data[0]

[1,
 14,
 22,
 16,
 43,
 530,
 973,
 1622,
 1385,
 65,
 458,
 4468,
 66,
 3941,
 4,
 173,
 36,
 256,
 5,
 25,
 100,
 43,
 838,
 112,
 50,
 670,
 2,
 9,
 35,
 480,
 284,
 5,
 150,
 4,
 172,
 112,
 167,
 2,
 336,
 385,
 39,
 4,
 172,
 4536,
 1111,
 17,
 546,
 38,
 13,
 447,
 4,
 192,
 50,
 16,
 6,
 147,
 2025,
 19,
 14,
 22,
 4,
 1920,
 4613,
 469,
 4,
 22,
 71,
 87,
 12,
 16,
 43,
 530,
 38,
 76,
 15,
 13,
 1247,
 4,
 22,
 17,
 515,
 17,
 12,
 16,
 626,
 18,
 2,
 5,
 62,
 386,
 12,
 8,
 316,
 8,
 106,
 5,
 4,
 2223,
 5244,
 16,
 480,
 66,
 3785,
 33,
 4,
 130,
 12,
 16,
 38,
 619,
 5,
 25,
 124,
 51,
 36,
 135,
 48,
 25,
 1415,
 33,
 6,
 22,
 12,
 215,
 28,
 77,
 52,
 5,
 14,
 407,
 16,
 82,
 2,
 8,
 4,
 107,
 117,
 5952,
 15,
 256,
 4,
 2,
 7,
 3766,
 5,
 723,
 36,
 71,
 43,
 530,
 476,
 26,
 400,
 317,
 46,
 7,
 4,
 2,
 1029,
 13,
 104,
 88,
 4,
 381,
 15,
 297,
 98,
 32,
 2071,
 56,
 26,
 141,
 6,
 194,
 7486,
 18,
 4,
 226,
 22,
 21,
 134,
 476,
 26,
 480,
 5,
 144,
 30,
 5535,
 18,

## The word directory is sorted by most frequent words. 

In [4]:
import operator

# word_dict is a dictionary mapping words to an integer index
word_dict = imdb.get_word_index()
sorted(word_dict.items(), key=operator.itemgetter(1))

[('the', 1),
 ('and', 2),
 ('a', 3),
 ('of', 4),
 ('to', 5),
 ('is', 6),
 ('br', 7),
 ('in', 8),
 ('it', 9),
 ('i', 10),
 ('this', 11),
 ('that', 12),
 ('was', 13),
 ('as', 14),
 ('for', 15),
 ('with', 16),
 ('movie', 17),
 ('but', 18),
 ('film', 19),
 ('on', 20),
 ('not', 21),
 ('you', 22),
 ('are', 23),
 ('his', 24),
 ('have', 25),
 ('he', 26),
 ('be', 27),
 ('one', 28),
 ('all', 29),
 ('at', 30),
 ('by', 31),
 ('an', 32),
 ('they', 33),
 ('who', 34),
 ('so', 35),
 ('from', 36),
 ('like', 37),
 ('her', 38),
 ('or', 39),
 ('just', 40),
 ('about', 41),
 ("it's", 42),
 ('out', 43),
 ('has', 44),
 ('if', 45),
 ('some', 46),
 ('there', 47),
 ('what', 48),
 ('good', 49),
 ('more', 50),
 ('when', 51),
 ('very', 52),
 ('up', 53),
 ('no', 54),
 ('time', 55),
 ('she', 56),
 ('even', 57),
 ('my', 58),
 ('would', 59),
 ('which', 60),
 ('only', 61),
 ('story', 62),
 ('really', 63),
 ('see', 64),
 ('their', 65),
 ('had', 66),
 ('can', 67),
 ('were', 68),
 ('me', 69),
 ('well', 70),
 ('than', 71),


## Review Decoding

In [6]:
# Reverse (word, index) => (index, word) mapping
reverse_word_index = dict([(value, key) for (key, value) in word_dict.items()])

# Decode the review; note that the indices were offset by 3
# because 0, 1 and 2 are reserved indices for "padding", "start of sequence", and "unknown".
decoded_review = ' '.join([reverse_word_index.get(i - 3, '?') for i in train_data[0]])

In [7]:
decoded_review

"? this film was just brilliant casting location scenery story direction everyone's really suited the part they played and you could just imagine being there robert ? is an amazing actor and now the same being director ? father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for ? and would recommend it to everyone to watch and the fly fishing was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also ? to the two little boy's that played the ? of norman and paul they were just brilliant children are often left out of the ? list i think because the stars that play them all grown up are such a big profile for the whole film but these children are amazing and should be praised for what they have done don't you th

## Vectorization: One-hot-encode

In [8]:
import numpy as np
from numpy import array

# The dimension has to match the number of words in training dataset.
def vectorize_sequences(sequences, dimension=10000):
    results = np.zeros((len(sequences), dimension)) # initialize the matrix with all 0
    for i, sequence in enumerate(sequences):
        #print(sequence)
        results[i, sequence] = 1.
    return results


## Matrix Vectorization Sample

In [9]:
sample_data = array([
    [1,3,8],
    [2,5,7]
])

vectorize_sequences(sample_data, 10)

array([[ 0.,  1.,  0.,  1.,  0.,  0.,  0.,  0.,  1.,  0.],
       [ 0.,  0.,  1.,  0.,  0.,  1.,  0.,  1.,  0.,  0.]])

## Text Data Vectorization

In [10]:
x_train = vectorize_sequences(train_data)

x_test = vectorize_sequences(test_data)

In [11]:
# Vectorized labels
y_train = np.asarray(train_labels).astype('float32')
y_test = np.asarray(test_labels).astype('float32')

## Build the Neural Network

In [12]:
from keras import models
from keras import layers

model = models.Sequential()
model.add(layers.Dense(16, activation='relu', input_shape=(10000,)))
model.add(layers.Dense(16, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

In [13]:
from keras import optimizers
from keras import metrics

model.compile(optimizer=optimizers.RMSprop(lr=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy']
             )

## Split Validation set from Training set

In [14]:
x_val = x_train[:10000]
x_train = x_train[10000:]

y_val = y_train[:10000]
y_train = y_train[10000:]

## Train and Test the model

In [16]:
history = model.fit(x_train,
                   y_train,
                   epochs=5,
                   batch_size=512,
                   validation_data=(x_val, y_val))

Train on 15000 samples, validate on 10000 samples
Epoch 1/5
15000/15000 [==============================] - 3s 190us/step - loss: 0.0032 - acc: 0.9999 - val_loss: 0.7221 - val_acc: 0.8633
Epoch 2/5
15000/15000 [==============================] - 3s 190us/step - loss: 0.0055 - acc: 0.9990 - val_loss: 0.7461 - val_acc: 0.8644
Epoch 3/5
15000/15000 [==============================] - 3s 186us/step - loss: 0.0021 - acc: 0.9999 - val_loss: 0.7816 - val_acc: 0.8630
Epoch 4/5
15000/15000 [==============================] - 3s 185us/step - loss: 0.0061 - acc: 0.9985 - val_loss: 0.8055 - val_acc: 0.8645
Epoch 5/5
15000/15000 [==============================] - 3s 214us/step - loss: 0.0013 - acc: 0.9999 - val_loss: 0.8286 - val_acc: 0.8638


In [ ]:
model.evaluate(x_test, y_test)

## Overfitting 

1. The training loss decreases with every epoch and the training accuracy increases with every epoch.
2. The validation loss and accuracy seem to peak at the fourth epoch
3. A model that performs better on the training data isn't necessarily a model that will do better on data it has never seen before.


In [17]:
model.predict(x_test[1:10])

array([[  1.00000000e+00],
       [  9.80072379e-01],
       [  9.49218750e-01],
       [  9.96803522e-01],
       [  9.99739707e-01],
       [  9.40192863e-02],
       [  3.73171048e-07],
       [  9.51588452e-01],
       [  1.00000000e+00]], dtype=float32)